In [11]:
trips = pd.read_csv('north_hourly_ridership.csv')

In [12]:
trips.head()

,stop_id,stop_sequence,hour,trips_in_hour
0,901736.0,1.0,0,2
1,901736.0,1.0,1,2
2,901736.0,1.0,2,1
3,901736.0,1.0,3,1
4,901736.0,1.0,4,2


In [13]:
hourly_proportions = pd.read_excel('HOURLY PROPORTIONS.xlsx')

In [14]:
hourly_proportions = hourly_proportions.rename(columns={'HOUR': 'hour'})

In [15]:
trips_with_proportions = trips.merge(hourly_proportions, on='hour', how='left')

In [1]:
import numpy as np

def generate_ridership_scenarios(df, n_scenarios=10):
    """
    Generate multiple ridership scenarios using triangular distribution.
    
    Parameters:
    - df: Original dataframe with baseline ridership
    - n_scenarios: Number of scenarios to generate
    
    Returns:
    - scenarios: List of numpy arrays, each containing ridership for all stops
    """
    scenarios = []
    baseline_ridership = df['Actual'].values
    
    for i in range(n_scenarios):
        # Triangular distribution: min=0.75*baseline, mode=baseline, max=1.25*baseline
        random_ridership = np.random.triangular(
            left=0.75 * baseline_ridership,
            mode=baseline_ridership,
            right=1.25 * baseline_ridership
        )
        scenarios.append(random_ridership)
    
    print(f"Generated {n_scenarios} scenarios")
    print(f"Baseline total ridership: {baseline_ridership.sum():.2f}")
    for i, scenario in enumerate(scenarios):
        print(f"  Scenario {i+1}: {scenario.sum():.2f} total ridership")
    
    return scenarios

In [2]:
def calculate_dwell_times_for_scenarios(df, scenarios, trips, hourly_proportions):
    """
    Calculate dwell times for all scenarios.
    
    Returns:
    - List of dwell time arrays, one for each scenario
    """
    scenario_dwell_times = []
    
    print(f"Calculating dwell times for {len(scenarios)} scenarios...")
    
    for idx, scenario_ridership in enumerate(scenarios):
        # Create temp df with this scenario's ridership
        temp_df = df.copy()
        temp_df['Actual'] = scenario_ridership
        
        # Calculate dwell times
        stop_ridership = temp_df[['stop_id', 'stop_sequence', 'Actual']].copy()
        stop_ridership = stop_ridership.rename(columns={'Actual': 'daily_ridership'})
        
        avg_dwell_times = calculate_dwell_times(trips, hourly_proportions, stop_ridership)
        
        # Convert to array ordered by stop index
        dwell_array = temp_df.set_index(['stop_id', 'stop_sequence']).index.map(avg_dwell_times).values
        scenario_dwell_times.append(dwell_array)
        
        print(f"  Scenario {idx+1}: Avg dwell time = {dwell_array.mean():.2f}s")
    
    return scenario_dwell_times

In [3]:
def run_stochastic_optimization(df, scenarios, scenario_dwell_times, total_distance_miles, min_stops=35):
    """
    Run stochastic optimization across multiple scenarios - LINEAR VERSION.
    """
    n_stops = len(df)
    n_scenarios = len(scenarios)
    
    # Parameters
    C_IVT = 17 / 3600
    C_walk = 30 / 3600
    WALK_SPEED = 1.4
    
    # Calculate baseline values
    baseline_run_time = df['run_time'].sum()
    baseline_n_stops = len(df)
    
    # Use FIXED acc/decel time (baseline value)
    baseline_accel_decel = 23.4 - (1.53 * baseline_n_stops / total_distance_miles)
    
    # Create model
    m = gp.Model("Stochastic_Bus_Stop_Optimization")
    m.setParam('OutputFlag', 1)
    
    # Decision variables
    x = m.addVars(n_stops, vtype=GRB.BINARY, name="x")
    
    # For each scenario, calculate its cost
    scenario_costs = []
    
    print(f"\nBuilding optimization model for {n_scenarios} scenarios...")
    
    for s in range(n_scenarios):
        scenario_ridership = scenarios[s]
        scenario_dwell = scenario_dwell_times[s]
        total_passengers = scenario_ridership.sum()
        
        # SIMPLIFIED LINEAR MODEL:
        # - Use baseline run time (no speed adjustment)
        # - Use baseline acc/decel time per stop
        # - Only dwell time varies with which stops are kept
        
        run_time_total = baseline_run_time
        dwell_time_total = gp.quicksum(scenario_dwell[i] * x[i] for i in range(n_stops))
        accel_decel_total = baseline_accel_decel * gp.quicksum(x[i] for i in range(n_stops))
        
        total_travel_time = run_time_total + dwell_time_total + accel_decel_total
        in_vehicle_cost = C_IVT * total_passengers * total_travel_time
        
        # Walk cost
        walk_penalties = []
        for i in range(n_stops):
            dist_to_prev = df.loc[i, 'distance_from_prev'] if i > 0 else float('inf')
            dist_to_next = df.loc[i, 'distance_to_next'] if i < n_stops-1 else float('inf')
            nearest_distance = min(dist_to_prev, dist_to_next)
            walk_time = nearest_distance / WALK_SPEED
            ridership = scenario_ridership[i]
            penalty = ridership * walk_time * C_walk
            walk_penalties.append(penalty)
        
        walk_cost = gp.quicksum(walk_penalties[i] * (1 - x[i]) for i in range(n_stops))
        
        scenario_cost = in_vehicle_cost + walk_cost
        scenario_costs.append(scenario_cost)
    
    # OBJECTIVE: Minimize AVERAGE cost
    avg_cost = gp.quicksum(scenario_costs) / n_scenarios
    m.setObjective(avg_cost, GRB.MINIMIZE)
    
    # Constraints
    m.addConstr(x[0] == 1, "keep_first")
    m.addConstr(x[n_stops-1] == 1, "keep_last")
    m.addConstr(gp.quicksum(x[i] for i in range(n_stops)) >= min_stops, "min_stops")
    
    MAX_WALK = 400
    for i in range(1, n_stops-1):
        neighbors = []
        if i > 0 and df.loc[i, 'distance_from_prev'] <= MAX_WALK:
            neighbors.append(i-1)
        if i < n_stops-1 and df.loc[i, 'distance_to_next'] <= MAX_WALK:
            neighbors.append(i+1)
        if len(neighbors) > 0:
            m.addConstr((1 - x[i]) <= gp.quicksum(x[j] for j in neighbors), f"max_walk_{i}")
    
    for i in range(1, n_stops-1):
        m.addConstr(x[i] + x[i+1] >= 1, f"no_consecutive_{i}")
    
    print("\nOptimizing...")
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        kept_indices = [i for i in range(n_stops) if x[i].X > 0.5]
        eliminated_indices = [i for i in range(n_stops) if x[i].X < 0.5]
        
        return {
            'kept_stop_indices': kept_indices,
            'eliminated_stop_indices': eliminated_indices,
            'avg_cost_across_scenarios': m.objVal,
            'n_kept_stops': len(kept_indices),
            'n_eliminated_stops': len(eliminated_indices),
            'scenarios': scenarios,
            'scenario_dwell_times': scenario_dwell_times
        }
    else:
        print("No optimal solution found!")
        return None

In [4]:
def calculate_dwell_times(trips_df, hourly_proportions_df, stop_ridership_df):
    """
    Calculate average dwell time for each stop based on current ridership.
    
    Parameters:
    - trips_df: DataFrame with columns [stop_id, stop_sequence, hour, trips_in_hour]
    - hourly_proportions_df: DataFrame with columns [hour, PERCENTAGE]
    - stop_ridership_df: DataFrame with columns [stop_id, stop_sequence, daily_ridership]
                         (daily_ridership will change as stops are redistributed)
    
    Returns:
    - Series with average dwell time per stop, indexed by [stop_id, stop_sequence]
    """
    
    # Merge trips with hourly proportions
    trips_with_proportions = trips_df.merge(
        hourly_proportions_df,
        on='hour',
        how='left'
    )
    
    # Merge with current ridership data
    ridership_analysis = trips_with_proportions.merge(
        stop_ridership_df[['stop_id', 'stop_sequence', 'daily_ridership']],
        on=['stop_id', 'stop_sequence'],
        how='left'
    )
    
    # Calculate hourly ridership
    ridership_analysis['hourly_ridership'] = (
        ridership_analysis['daily_ridership'] * ridership_analysis['PERCENTAGE']
    )
    
    # Calculate ridership per bus
    ridership_analysis['ridership_per_bus'] = (
        ridership_analysis['hourly_ridership'] / ridership_analysis['trips_in_hour']
    )
    
    # Calculate dwell time: 5 + 2.75 * ridership_per_bus
    ridership_analysis['dwell_time'] = 5 + (2.75 * ridership_analysis['ridership_per_bus'])
    
    # Calculate average dwell time per stop
    avg_dwell_per_stop = (
        ridership_analysis.groupby(['stop_id', 'stop_sequence'])['dwell_time']
        .mean()
        .sort_index(level='stop_sequence')
    )
    
    return avg_dwell_per_stop

In [5]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Load data
filename = 'north_riders_optimization.csv'
df = pd.read_csv(filename)

# Filter to only route stops (first to last stop in sequence)
df = df[df['stop_name'].notna()].reset_index(drop=True)
max_sequence = df['stop_sequence'].max()
df = df[df['stop_sequence'] <= max_sequence].reset_index(drop=True)

n_stops = len(df)

In [7]:
def calculate_accel_decel_time(stops_df, kept_stop_indices, total_route_distance_miles):
    """
    Calculate acceleration/deceleration time based on stops per mile.
    
    Parameters:
    - stops_df: DataFrame with distance information (already filtered to route stops only)
    - kept_stop_indices: List of indices for stops that are kept open
    - total_route_distance_miles: Total route distance in miles (constant)
    
    Returns:
    - accel_decel_time: Time in seconds per stop (T = 23.4 - 1.53X)
    - stops_per_mile: Number of stops per mile
    """
    
    # Number of kept stops
    num_kept_stops = len(kept_stop_indices)
    
    # Calculate stops per mile (using total route distance)
    stops_per_mile = num_kept_stops / total_route_distance_miles
    
    # Calculate acc/decel time using formula: T = 23.4 - 1.53X
    accel_decel_time = 23.4 - (1.53 * stops_per_mile)
    
    return accel_decel_time, stops_per_mile

In [8]:
# Calculate total route distance (do this once at the beginning)
total_distance_meters = df['distance_to_next'].sum()
total_distance_miles = total_distance_meters / 1609.34

print(f"Total route distance: {total_distance_miles:.2f} miles")

# Now test the function with this distance
all_stop_indices = list(range(len(df)))
accel_decel, spm = calculate_accel_decel_time(df, all_stop_indices, total_distance_miles)

print(f"Number of stops: {len(all_stop_indices)}")
print(f"Stops per mile: {spm:.2f}")
print(f"Accel/Decel time: {accel_decel:.2f} seconds")

print(f"\nYour original accel_decel_time: {df['accel_decel_time'].iloc[0]:.2f}")
print(f"Match: {abs(accel_decel - df['accel_decel_time'].iloc[0]) < 0.01}")

Total route distance: 6.55 miles
Number of stops: 53
Stops per mile: 8.09
Accel/Decel time: 11.03 seconds

Your original accel_decel_time: 11.03
Match: True


In [9]:
def redistribute_ridership(stops_df, kept_stop_indices):
    """
    Redistribute ridership from closed stops to nearest open stops.
    
    Parameters:
    - stops_df: DataFrame with stop information
    - kept_stop_indices: List of indices for stops that are kept open
    
    Returns:
    - DataFrame with updated ridership for each stop
    """
    
    # Create a copy with a new column for updated ridership
    updated_df = stops_df.copy()
    updated_df['daily_ridership'] = updated_df['Actual'].copy()
    
    # Get list of closed stops
    all_indices = set(range(len(stops_df)))
    closed_stop_indices = list(all_indices - set(kept_stop_indices))
    
    # For each closed stop, find nearest open stop and transfer ridership
    for closed_idx in closed_stop_indices:
        closed_ridership = updated_df.loc[closed_idx, 'Actual']
        
        # Find nearest open stop
        min_distance = float('inf')
        nearest_open_idx = None
        
        # Check all kept stops and find the one with minimum distance
        for open_idx in kept_stop_indices:
            if open_idx == closed_idx:
                continue
                
            # Calculate distance between closed and open stop
            if open_idx < closed_idx:
                # Open stop is before closed stop - sum distances going forward
                distance = stops_df.loc[open_idx:closed_idx-1, 'distance_to_next'].sum()
            else:
                # Open stop is after closed stop - sum distances going backward
                distance = stops_df.loc[closed_idx:open_idx-1, 'distance_to_next'].sum()
            
            if distance < min_distance:
                min_distance = distance
                nearest_open_idx = open_idx
        
        # Transfer ridership to nearest open stop
        if nearest_open_idx is not None:
            updated_df.loc[nearest_open_idx, 'daily_ridership'] += closed_ridership
            updated_df.loc[closed_idx, 'daily_ridership'] = 0  # Set closed stop to 0
    
    return updated_df[['stop_id', 'stop_sequence', 'daily_ridership']]

In [16]:
# Run the stochastic optimization across multiple scenarios
print("\n" + "="*70)
print("STOCHASTIC OPTIMIZATION ACROSS MULTIPLE SCENARIOS")
print("="*70)

# Calculate total distance (if not already done)
total_distance_meters = df['distance_to_next'].sum()
total_distance_miles = total_distance_meters / 1609.34

# Step 1: Generate 10 ridership scenarios
print("\nStep 1: Generating scenarios...")
scenarios = generate_ridership_scenarios(df, n_scenarios=10)

# Step 2: Calculate dwell times for all scenarios
print("\nStep 2: Calculating dwell times for all scenarios...")
scenario_dwell_times = calculate_dwell_times_for_scenarios(df, scenarios, trips, hourly_proportions)

# Step 3: Run stochastic optimization
print("\nStep 3: Running stochastic optimization...")
stochastic_solution = run_stochastic_optimization(
    df=df,
    scenarios=scenarios,
    scenario_dwell_times=scenario_dwell_times,
    total_distance_miles=total_distance_miles,
    min_stops=35
)

# Print summary of results
print("\n" + "="*70)
print("STOCHASTIC OPTIMIZATION RESULTS")
print("="*70)
print(f"Stops kept: {stochastic_solution['n_kept_stops']} out of {len(df)}")
print(f"Stops eliminated: {stochastic_solution['n_eliminated_stops']}")
print(f"Average cost across 10 scenarios: ${stochastic_solution['avg_cost_across_scenarios']:,.2f}")

# Show which stops are kept/eliminated
print(f"\nKept stop indices: {stochastic_solution['kept_stop_indices'][:10]}, ...")
print(f"Eliminated stop indices: {stochastic_solution['eliminated_stop_indices']}")

print("\n" + "="*70)


STOCHASTIC OPTIMIZATION ACROSS MULTIPLE SCENARIOS

Step 1: Generating scenarios...
Generated 10 scenarios
Baseline total ridership: 3200.35
  Scenario 1: 3131.85 total ridership
  Scenario 2: 3205.72 total ridership
  Scenario 3: 3250.15 total ridership
  Scenario 4: 3094.29 total ridership
  Scenario 5: 3135.17 total ridership
  Scenario 6: 3225.46 total ridership
  Scenario 7: 3169.14 total ridership
  Scenario 8: 3122.33 total ridership
  Scenario 9: 3207.43 total ridership
  Scenario 10: 3250.07 total ridership

Step 2: Calculating dwell times for all scenarios...
Calculating dwell times for 10 scenarios...
  Scenario 1: Avg dwell time = 6.69s
  Scenario 2: Avg dwell time = 6.73s
  Scenario 3: Avg dwell time = 6.76s
  Scenario 4: Avg dwell time = 6.67s
  Scenario 5: Avg dwell time = 6.69s
  Scenario 6: Avg dwell time = 6.74s
  Scenario 7: Avg dwell time = 6.71s
  Scenario 8: Avg dwell time = 6.69s
  Scenario 9: Avg dwell time = 6.73s
  Scenario 10: Avg dwell time = 6.76s

Step 3: 

In [18]:
def calculate_adjusted_run_time(baseline_run_time, baseline_n_stops, optimized_n_stops, adjustment_factor=0.2):
    """
    Calculate adjusted run time based on speed increase from eliminating stops.
    
    Parameters:
    - baseline_run_time: Original total run time
    - baseline_n_stops: Number of stops in baseline
    - optimized_n_stops: Number of stops in optimized solution
    - adjustment_factor: Maximum speed improvement factor (default 0.10 = 10%)
    
    Logic: If we eliminate X% of stops, we can increase speed by (X% * adjustment_factor)
    For example: Eliminate 50% of stops → speed increases by 50% * 10% = 5%
    → run time decreases by 5%
    
    Returns:
    - adjusted_run_time: New run time accounting for speed increase
    """
    
   # Use baseline run time (no speed adjustment in optimization to keep it linear)
    adjusted_run_time = baseline_run_time
    
    return adjusted_run_time

In [26]:
print("\n" + "="*80)
print("BASELINE vs STOCHASTIC OPTIMIZATION - DETAILED COMPARISON")
print("="*80)

# ============================================================================
# BASELINE METRICS (from original df)
# ============================================================================
baseline_n_stops = len(df)
baseline_daily_passengers = df['Actual'].sum()
baseline_run_time = df['run_time'].sum()
baseline_dwell_time = df['avg_dwell_time'].sum()
baseline_accel_decel_per_stop = df['accel_decel_time'].iloc[0]
baseline_accel_decel_total = baseline_accel_decel_per_stop * baseline_n_stops
baseline_total_time = baseline_run_time + baseline_dwell_time + baseline_accel_decel_total

C_IVT = 17 / 3600
baseline_cost = C_IVT * baseline_daily_passengers * baseline_total_time

# ============================================================================
# STOCHASTIC OPTIMIZED METRICS (with redistribution)
# ============================================================================
kept_indices = stochastic_solution['kept_stop_indices']
eliminated_indices = stochastic_solution['eliminated_stop_indices']
optimized_n_stops = len(kept_indices)

# Step 1: Redistribute ridership from closed stops to nearest open stops
redistributed = redistribute_ridership(df, kept_indices)
temp_df = df.copy()
temp_df['Actual'] = redistributed['daily_ridership'].values

# Step 2: Recalculate dwell times with redistributed ridership
stop_ridership = temp_df[['stop_id', 'stop_sequence', 'Actual']].rename(columns={'Actual': 'daily_ridership'})
new_dwell_times = calculate_dwell_times(trips, hourly_proportions, stop_ridership)
temp_df['avg_dwell_time'] = temp_df.set_index(['stop_id', 'stop_sequence']).index.map(new_dwell_times)

# Step 3: Calculate new acc/decel time based on new stops per mile
new_accel_decel, new_stops_per_mile = calculate_accel_decel_time(temp_df, kept_indices, total_distance_miles)
new_accel_decel = float(new_accel_decel)

# Step 4: Sum up optimized times (only for kept stops)
optimized_run_time = baseline_run_time  # Run time stays the same
optimized_dwell_time = temp_df.iloc[kept_indices]['avg_dwell_time'].sum()
optimized_accel_decel_total = new_accel_decel * optimized_n_stops
optimized_total_time = optimized_run_time + optimized_dwell_time + optimized_accel_decel_total

optimized_cost = C_IVT * baseline_daily_passengers * optimized_total_time

# ============================================================================
# PRINT COMPARISON
# ============================================================================

print("\nSTOPS")
print("-"*80)
print(f"Baseline:          {baseline_n_stops} stops")
print(f"Optimized:         {optimized_n_stops} stops")
print(f"Eliminated:        {baseline_n_stops - optimized_n_stops} stops ({(baseline_n_stops - optimized_n_stops)/baseline_n_stops*100:.1f}%)")

print("\nTRAVEL TIME COMPONENTS")
print("-"*80)
print(f"Run Time:")
print(f"  Baseline:        {baseline_run_time:.1f}s ({baseline_run_time/60:.2f} min)")
print(f"  Optimized:       {optimized_run_time:.1f}s ({optimized_run_time/60:.2f} min)")
print(f"  Change:          0s (stays the same)")

print(f"\nDwell Time (total across all stops):")
print(f"  Baseline:        {baseline_dwell_time:.1f}s ({baseline_dwell_time/60:.2f} min)")
print(f"  Optimized:       {optimized_dwell_time:.1f}s ({optimized_dwell_time/60:.2f} min)")
print(f"  Savings:         {baseline_dwell_time - optimized_dwell_time:.1f}s ({(baseline_dwell_time - optimized_dwell_time)/60:.2f} min)")

print(f"\nAcc/Decel Time (total):")
print(f"  Baseline:        {baseline_accel_decel_total:.1f}s ({baseline_accel_decel_per_stop:.2f}s per stop × {baseline_n_stops} stops)")
print(f"  Optimized:       {optimized_accel_decel_total:.1f}s ({new_accel_decel:.2f}s per stop × {optimized_n_stops} stops)")
print(f"  Savings:         {baseline_accel_decel_total - optimized_accel_decel_total:.1f}s ({(baseline_accel_decel_total - optimized_accel_decel_total)/60:.2f} min)")

print(f"\nTOTAL TRAVEL TIME:")
print(f"  Baseline:        {baseline_total_time:.1f}s ({baseline_total_time/60:.2f} min)")
print(f"  Optimized:       {optimized_total_time:.1f}s ({optimized_total_time/60:.2f} min)")
print(f"  Savings:         {baseline_total_time - optimized_total_time:.1f}s ({(baseline_total_time - optimized_total_time)/60:.2f} min)")
print(f"  Improvement:     {(baseline_total_time - optimized_total_time)/baseline_total_time*100:.1f}%")

print("\nCOSTS")
print("-"*80)
print(f"Daily cost:")
print(f"  Baseline:        ${baseline_cost:,.2f}")
print(f"  Optimized:       ${optimized_cost:,.2f}")
print(f"  Savings:         ${baseline_cost - optimized_cost:,.2f} ({(baseline_cost - optimized_cost)/baseline_cost*100:.1f}%)")

print(f"\nAnnual cost:")
print(f"  Baseline:        ${baseline_cost * 365:,.2f}")
print(f"  Optimized:       ${optimized_cost * 365:,.2f}")
print(f"  Savings:         ${(baseline_cost - optimized_cost) * 365:,.2f}")

print("\nROUTE CHARACTERISTICS")
print("-"*80)
baseline_spm = baseline_n_stops / total_distance_miles
optimized_spm = optimized_n_stops / total_distance_miles
print(f"Stops per mile:    {baseline_spm:.2f} -> {optimized_spm:.2f}")

baseline_speed_mph = total_distance_miles / (baseline_total_time / 3600)
optimized_speed_mph = total_distance_miles / (optimized_total_time / 3600)
print(f"Average speed:     {baseline_speed_mph:.2f} mph -> {optimized_speed_mph:.2f} mph")
print(f"Speed improvement: {(optimized_speed_mph - baseline_speed_mph)/baseline_speed_mph*100:.1f}%")

print("\n" + "="*80)
print(f"SUMMARY: Eliminating {baseline_n_stops - optimized_n_stops} stops saves {(baseline_total_time - optimized_total_time)/60:.1f} minutes per trip")
print(f"         and ${(baseline_cost - optimized_cost) * 365:,.2f} per year")
print("="*80)


BASELINE vs STOCHASTIC OPTIMIZATION - DETAILED COMPARISON

STOPS
--------------------------------------------------------------------------------
Baseline:          53 stops
Optimized:         35 stops
Eliminated:        18 stops (34.0%)

TRAVEL TIME COMPONENTS
--------------------------------------------------------------------------------
Run Time:
  Baseline:        3022.3s (50.37 min)
  Optimized:       3022.3s (50.37 min)
  Change:          0s (stays the same)

Dwell Time (total across all stops):
  Baseline:        356.7s (5.94 min)
  Optimized:       266.7s (4.44 min)
  Savings:         90.0s (1.50 min)

Acc/Decel Time (total):
  Baseline:        584.5s (11.03s per stop × 53 stops)
  Optimized:       533.1s (15.23s per stop × 35 stops)
  Savings:         51.5s (0.86 min)

TOTAL TRAVEL TIME:
  Baseline:        3963.5s (66.06 min)
  Optimized:       3822.0s (63.70 min)
  Savings:         141.5s (2.36 min)
  Improvement:     3.6%

COSTS
--------------------------------------------

In [27]:
import pandas as pd

print("\n" + "="*80)
print("STORING STOCHASTIC OPTIMIZATION SCENARIOS AND RESULTS")
print("="*80)

# ============================================================================
# 1. STORE ALL 10 SCENARIOS (ridership data)
# ============================================================================

# Create a dataframe with all scenarios
scenarios_data = []

for scenario_idx, scenario_ridership in enumerate(stochastic_solution['scenarios'], 1):
    for stop_idx, ridership in enumerate(scenario_ridership):
        scenarios_data.append({
            'scenario_number': scenario_idx,
            'stop_index': stop_idx,
            'stop_id': df.loc[stop_idx, 'stop_id'],
            'stop_sequence': df.loc[stop_idx, 'stop_sequence'],
            'stop_name': df.loc[stop_idx, 'stop_name'],
            'baseline_ridership': df.loc[stop_idx, 'Actual'],
            'scenario_ridership': ridership
        })

scenarios_df = pd.DataFrame(scenarios_data)

print(f"\nCreated scenarios dataframe with {len(scenarios_df)} rows")
print(f"Shape: {scenarios_df.shape}")
print("\nSample:")
print(scenarios_df.head(10))

# ============================================================================
# 2. STORE SCENARIO DWELL TIMES
# ============================================================================

dwell_times_data = []

for scenario_idx, scenario_dwell in enumerate(stochastic_solution['scenario_dwell_times'], 1):
    for stop_idx, dwell_time in enumerate(scenario_dwell):
        dwell_times_data.append({
            'scenario_number': scenario_idx,
            'stop_index': stop_idx,
            'stop_id': df.loc[stop_idx, 'stop_id'],
            'stop_sequence': df.loc[stop_idx, 'stop_sequence'],
            'baseline_dwell_time': df.loc[stop_idx, 'avg_dwell_time'],
            'scenario_dwell_time': dwell_time
        })

dwell_times_df = pd.DataFrame(dwell_times_data)

print(f"\nCreated dwell times dataframe with {len(dwell_times_df)} rows")
print(f"Shape: {dwell_times_df.shape}")

# ============================================================================
# 3. STORE OPTIMIZATION SOLUTION
# ============================================================================

solution_summary = pd.DataFrame([{
    'total_stops': len(df),
    'kept_stops': stochastic_solution['n_kept_stops'],
    'eliminated_stops': stochastic_solution['n_eliminated_stops'],
    'avg_cost_across_scenarios': stochastic_solution['avg_cost_across_scenarios'],
    'baseline_cost': baseline_cost,
    'cost_savings': baseline_cost - optimized_cost,
    'percent_savings': (baseline_cost - optimized_cost) / baseline_cost * 100,
    'baseline_travel_time': baseline_total_time,
    'optimized_travel_time': optimized_total_time,
    'time_savings_seconds': baseline_total_time - optimized_total_time,
    'time_savings_minutes': (baseline_total_time - optimized_total_time) / 60
}])

print("\nOptimization Solution Summary:")
print(solution_summary.T)

# ============================================================================
# 4. STORE STOP-LEVEL RESULTS
# ============================================================================

stop_results = []

for stop_idx in range(len(df)):
    stop_results.append({
        'stop_index': stop_idx,
        'stop_id': df.loc[stop_idx, 'stop_id'],
        'stop_sequence': df.loc[stop_idx, 'stop_sequence'],
        'stop_name': df.loc[stop_idx, 'stop_name'],
        'status': 'Open' if stop_idx in kept_indices else 'Closed',
        'baseline_ridership': df.loc[stop_idx, 'Actual'],
        'baseline_dwell_time': df.loc[stop_idx, 'avg_dwell_time'],
        'distance_to_next': df.loc[stop_idx, 'distance_to_next'],
        'distance_from_prev': df.loc[stop_idx, 'distance_from_prev']
    })

stop_results_df = pd.DataFrame(stop_results)

print(f"\nCreated stop results dataframe with {len(stop_results_df)} rows")
print("\nStops by Status:")
print(stop_results_df['status'].value_counts())

# ============================================================================
# 5. EXPORT ALL TO CSV
# ============================================================================

print("\n" + "="*80)
print("EXPORTING TO CSV FILES")
print("="*80)

# Export all dataframes
scenarios_df.to_csv('stochastic_scenarios_ridership.csv', index=False)
print("✓ Saved: stochastic_scenarios_ridership.csv")

dwell_times_df.to_csv('stochastic_scenarios_dwell_times.csv', index=False)
print("✓ Saved: stochastic_scenarios_dwell_times.csv")

solution_summary.to_csv('optimization_solution_summary.csv', index=False)
print("✓ Saved: optimization_solution_summary.csv")

stop_results_df.to_csv('stop_results.csv', index=False)
print("✓ Saved: stop_results.csv")

# ============================================================================
# 6. CREATE A COMPREHENSIVE EXCEL FILE WITH MULTIPLE SHEETS
# ============================================================================

print("\nCreating comprehensive Excel file with all results...")

with pd.ExcelWriter('stochastic_optimization_results.xlsx', engine='openpyxl') as writer:
    # Sheet 1: Summary
    solution_summary.to_excel(writer, sheet_name='Summary', index=False)
    
    # Sheet 2: Stop Results
    stop_results_df.to_excel(writer, sheet_name='Stop_Results', index=False)
    
    # Sheet 3: All Scenarios Ridership
    scenarios_df.to_excel(writer, sheet_name='Scenarios_Ridership', index=False)
    
    # Sheet 4: All Scenarios Dwell Times
    dwell_times_df.to_excel(writer, sheet_name='Scenarios_Dwell_Times', index=False)
    
    # Sheet 5: Kept Stops Detail
    kept_stops_detail = stop_results_df[stop_results_df['status'] == 'Open'].copy()
    kept_stops_detail.to_excel(writer, sheet_name='Kept_Stops', index=False)
    
    # Sheet 6: Eliminated Stops Detail
    eliminated_stops_detail = stop_results_df[stop_results_df['status'] == 'Closed'].copy()
    eliminated_stops_detail.to_excel(writer, sheet_name='Eliminated_Stops', index=False)

print("✓ Saved: stochastic_optimization_results.xlsx (with 6 sheets)")

print("\n" + "="*80)
print("ALL DATA SAVED SUCCESSFULLY!")
print("="*80)

print("\nFiles created:")
print("  1. stochastic_scenarios_ridership.csv - All 10 scenarios' ridership data")
print("  2. stochastic_scenarios_dwell_times.csv - All 10 scenarios' dwell times")
print("  3. optimization_solution_summary.csv - High-level summary of results")
print("  4. stop_results.csv - Stop-by-stop results (open/closed)")
print("  5. stochastic_optimization_results.xlsx - Comprehensive Excel with all sheets")

# ============================================================================
# 7. DISPLAY SUMMARY STATISTICS
# ============================================================================

print("\n" + "="*80)
print("SCENARIO STATISTICS")
print("="*80)

print("\nRidership across 10 scenarios:")
scenario_totals = scenarios_df.groupby('scenario_number')['scenario_ridership'].sum()
print(f"  Mean total ridership: {scenario_totals.mean():.2f}")
print(f"  Std deviation: {scenario_totals.std():.2f}")
print(f"  Min: {scenario_totals.min():.2f}")
print(f"  Max: {scenario_totals.max():.2f}")

print("\nDwell time across 10 scenarios:")
scenario_dwell = dwell_times_df.groupby('scenario_number')['scenario_dwell_time'].mean()
print(f"  Mean avg dwell time: {scenario_dwell.mean():.2f}s")
print(f"  Std deviation: {scenario_dwell.std():.2f}s")
print(f"  Min: {scenario_dwell.min():.2f}s")
print(f"  Max: {scenario_dwell.max():.2f}s")

print("\n" + "="*80)


STORING STOCHASTIC OPTIMIZATION SCENARIOS AND RESULTS

Created scenarios dataframe with 530 rows
Shape: (530, 7)

Sample:
   scenario_number  stop_index   stop_id  stop_sequence  \
0                1           0  901736.0            1.0   
1                1           1  303710.0            2.0   
2                1           2  303711.0            3.0   
3                1           3  303712.0            4.0   
4                1           4  303713.0            5.0   
5                1           5  303714.0            6.0   
6                1           6  306974.0            7.0   
7                1           7  306322.0            8.0   
8                1           8  303717.0            9.0   
9                1           9  303718.0           10.0   

                   stop_name  baseline_ridership  scenario_ridership  
0        LINCOLN RD/OCEAN AV          203.803279          220.162353  
1  EMPIRE BLVD/WASHINGTON AV           35.028689           32.843087  
2     EMPIRE B

In [28]:
import pandas as pd

print("\n" + "="*80)
print("STORING COMPREHENSIVE STOCHASTIC OPTIMIZATION RESULTS")
print("="*80)

# ============================================================================
# PREPARE OPTIMIZED DATA (with redistribution)
# ============================================================================

kept_indices = stochastic_solution['kept_stop_indices']
eliminated_indices = stochastic_solution['eliminated_stop_indices']

# Redistribute ridership
redistributed = redistribute_ridership(df, kept_indices)
temp_df = df.copy()
temp_df['Actual'] = redistributed['daily_ridership'].values

# Recalculate dwell times with redistributed ridership
stop_ridership = temp_df[['stop_id', 'stop_sequence', 'Actual']].rename(columns={'Actual': 'daily_ridership'})
optimized_dwell_times = calculate_dwell_times(trips, hourly_proportions, stop_ridership)
temp_df['avg_dwell_time'] = temp_df.set_index(['stop_id', 'stop_sequence']).index.map(optimized_dwell_times)

# Calculate optimized acc/decel time
optimized_accel_decel, optimized_stops_per_mile = calculate_accel_decel_time(temp_df, kept_indices, total_distance_miles)
optimized_accel_decel = float(optimized_accel_decel)

# Calculate walking cost for eliminated stops
C_walk = 30 / 3600
WALK_SPEED = 1.4
total_walk_cost = 0

walk_costs_by_stop = {}
for i in eliminated_indices:
    dist_to_prev = df.loc[i, 'distance_from_prev'] if i > 0 else float('inf')
    dist_to_next = df.loc[i, 'distance_to_next'] if i < len(df)-1 else float('inf')
    walk_distance = min(dist_to_prev, dist_to_next)
    walk_time = walk_distance / WALK_SPEED
    ridership = df.loc[i, 'Actual']
    walk_cost = ridership * walk_time * C_walk
    walk_costs_by_stop[i] = walk_cost
    total_walk_cost += walk_cost

# ============================================================================
# CREATE COMPREHENSIVE STOP-BY-STOP RESULTS
# ============================================================================

comprehensive_results = []

for stop_idx in range(len(df)):
    stop_status = 'Open' if stop_idx in kept_indices else 'Closed'
    
    comprehensive_results.append({
        'stop_index': stop_idx,
        'stop_id': df.loc[stop_idx, 'stop_id'],
        'stop_sequence': df.loc[stop_idx, 'stop_sequence'],
        'stop_name': df.loc[stop_idx, 'stop_name'],
        'status': stop_status,
        
        # Ridership
        'baseline_ridership': df.loc[stop_idx, 'Actual'],
        'optimized_ridership': temp_df.loc[stop_idx, 'Actual'],
        'ridership_change': temp_df.loc[stop_idx, 'Actual'] - df.loc[stop_idx, 'Actual'],
        
        # Dwell Time
        'baseline_dwell_time': df.loc[stop_idx, 'avg_dwell_time'],
        'optimized_dwell_time': temp_df.loc[stop_idx, 'avg_dwell_time'],
        'dwell_time_change': temp_df.loc[stop_idx, 'avg_dwell_time'] - df.loc[stop_idx, 'avg_dwell_time'],
        
        # Acc/Decel Time
        'baseline_accel_decel': df.loc[stop_idx, 'accel_decel_time'],
        'optimized_accel_decel': optimized_accel_decel if stop_status == 'Open' else 0,
        
        # Walk cost (only for closed stops)
        'walk_cost_daily': walk_costs_by_stop.get(stop_idx, 0),
        'walk_distance_to_nearest': min(df.loc[stop_idx, 'distance_from_prev'] if stop_idx > 0 else float('inf'),
                                         df.loc[stop_idx, 'distance_to_next'] if stop_idx < len(df)-1 else float('inf')) if stop_status == 'Closed' else 0,
        
        # Distance info
        'distance_to_next': df.loc[stop_idx, 'distance_to_next'],
        'distance_from_prev': df.loc[stop_idx, 'distance_from_prev']
    })

comprehensive_df = pd.DataFrame(comprehensive_results)

print("\nComprehensive Results DataFrame Created")
print(f"Shape: {comprehensive_df.shape}")
print("\nFirst 10 rows:")
print(comprehensive_df.head(10))

# ============================================================================
# CALCULATE SUMMARY STATISTICS
# ============================================================================

print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)

# Overall summary
total_stops = len(df)
open_stops = len(kept_indices)
closed_stops = len(eliminated_indices)

print(f"\nSTOPS:")
print(f"  Total stops:        {total_stops}")
print(f"  Open stops:         {open_stops}")
print(f"  Closed stops:       {closed_stops}")
print(f"  Percent eliminated: {closed_stops/total_stops*100:.1f}%")

# Ridership summary
total_baseline_ridership = comprehensive_df['baseline_ridership'].sum()
total_optimized_ridership = comprehensive_df['optimized_ridership'].sum()

print(f"\nRIDERSHIP:")
print(f"  Baseline total:     {total_baseline_ridership:.2f}")
print(f"  Optimized total:    {total_optimized_ridership:.2f}")
print(f"  (Should be equal - just redistributed)")

# Ridership at open stops
open_stops_baseline_ridership = comprehensive_df[comprehensive_df['status'] == 'Open']['baseline_ridership'].sum()
open_stops_optimized_ridership = comprehensive_df[comprehensive_df['status'] == 'Open']['optimized_ridership'].sum()

print(f"\n  At open stops:")
print(f"    Baseline:         {open_stops_baseline_ridership:.2f}")
print(f"    Optimized:        {open_stops_optimized_ridership:.2f}")
print(f"    Increase:         {open_stops_optimized_ridership - open_stops_baseline_ridership:.2f}")

# Dwell time summary
total_baseline_dwell = comprehensive_df['baseline_dwell_time'].sum()
total_optimized_dwell = comprehensive_df[comprehensive_df['status'] == 'Open']['optimized_dwell_time'].sum()

print(f"\nDWELL TIME:")
print(f"  Baseline total:     {total_baseline_dwell:.1f}s ({total_baseline_dwell/60:.2f} min)")
print(f"  Optimized total:    {total_optimized_dwell:.1f}s ({total_optimized_dwell/60:.2f} min)")
print(f"  Savings:            {total_baseline_dwell - total_optimized_dwell:.1f}s ({(total_baseline_dwell - total_optimized_dwell)/60:.2f} min)")

# Acc/Decel summary
baseline_accel_decel_per_stop = df['accel_decel_time'].iloc[0]
baseline_accel_decel_total = baseline_accel_decel_per_stop * total_stops
optimized_accel_decel_total = optimized_accel_decel * open_stops

print(f"\nACC/DECEL TIME:")
print(f"  Baseline:           {baseline_accel_decel_total:.1f}s ({baseline_accel_decel_per_stop:.2f}s × {total_stops} stops)")
print(f"  Optimized:          {optimized_accel_decel_total:.1f}s ({optimized_accel_decel:.2f}s × {open_stops} stops)")
print(f"  Savings:            {baseline_accel_decel_total - optimized_accel_decel_total:.1f}s")

# Travel time summary
baseline_run_time = df['run_time'].sum()
baseline_total_time = baseline_run_time + total_baseline_dwell + baseline_accel_decel_total
optimized_total_time = baseline_run_time + total_optimized_dwell + optimized_accel_decel_total

print(f"\nTOTAL TRAVEL TIME:")
print(f"  Baseline:           {baseline_total_time:.1f}s ({baseline_total_time/60:.2f} min)")
print(f"  Optimized:          {optimized_total_time:.1f}s ({optimized_total_time/60:.2f} min)")
print(f"  Savings:            {baseline_total_time - optimized_total_time:.1f}s ({(baseline_total_time - optimized_total_time)/60:.2f} min)")
print(f"  Improvement:        {(baseline_total_time - optimized_total_time)/baseline_total_time*100:.1f}%")

# Cost summary
C_IVT = 17 / 3600
baseline_in_vehicle_cost = C_IVT * total_baseline_ridership * baseline_total_time
baseline_walk_cost = 0
baseline_total_cost = baseline_in_vehicle_cost + baseline_walk_cost

optimized_in_vehicle_cost = C_IVT * total_optimized_ridership * optimized_total_time
optimized_walk_cost = total_walk_cost
optimized_total_cost = optimized_in_vehicle_cost + optimized_walk_cost

print(f"\nCOSTS:")
print(f"  Baseline:")
print(f"    In-vehicle cost:  ${baseline_in_vehicle_cost:,.2f}")
print(f"    Walk cost:        ${baseline_walk_cost:,.2f}")
print(f"    Total daily:      ${baseline_total_cost:,.2f}")
print(f"\n  Optimized:")
print(f"    In-vehicle cost:  ${optimized_in_vehicle_cost:,.2f}")
print(f"    Walk cost:        ${optimized_walk_cost:,.2f}")
print(f"    Total daily:      ${optimized_total_cost:,.2f}")
print(f"\n  Savings:")
print(f"    Daily:            ${baseline_total_cost - optimized_total_cost:,.2f} ({(baseline_total_cost - optimized_total_cost)/baseline_total_cost*100:.1f}%)")
print(f"    Annual:           ${(baseline_total_cost - optimized_total_cost) * 365:,.2f}")

# Create summary dataframe
summary_stats = pd.DataFrame([{
    'metric': 'Stops',
    'baseline': total_stops,
    'optimized': open_stops,
    'change': -(closed_stops),
    'percent_change': -closed_stops/total_stops*100
}, {
    'metric': 'Total Ridership',
    'baseline': total_baseline_ridership,
    'optimized': total_optimized_ridership,
    'change': 0,
    'percent_change': 0
}, {
    'metric': 'Dwell Time (seconds)',
    'baseline': total_baseline_dwell,
    'optimized': total_optimized_dwell,
    'change': -(total_baseline_dwell - total_optimized_dwell),
    'percent_change': -(total_baseline_dwell - total_optimized_dwell)/total_baseline_dwell*100
}, {
    'metric': 'Acc/Decel Time (seconds)',
    'baseline': baseline_accel_decel_total,
    'optimized': optimized_accel_decel_total,
    'change': -(baseline_accel_decel_total - optimized_accel_decel_total),
    'percent_change': -(baseline_accel_decel_total - optimized_accel_decel_total)/baseline_accel_decel_total*100
}, {
    'metric': 'Total Travel Time (seconds)',
    'baseline': baseline_total_time,
    'optimized': optimized_total_time,
    'change': -(baseline_total_time - optimized_total_time),
    'percent_change': -(baseline_total_time - optimized_total_time)/baseline_total_time*100
}, {
    'metric': 'In-Vehicle Cost ($)',
    'baseline': baseline_in_vehicle_cost,
    'optimized': optimized_in_vehicle_cost,
    'change': -(baseline_in_vehicle_cost - optimized_in_vehicle_cost),
    'percent_change': -(baseline_in_vehicle_cost - optimized_in_vehicle_cost)/baseline_in_vehicle_cost*100
}, {
    'metric': 'Walk Cost ($)',
    'baseline': baseline_walk_cost,
    'optimized': optimized_walk_cost,
    'change': optimized_walk_cost,
    'percent_change': float('inf') if baseline_walk_cost == 0 else 0
}, {
    'metric': 'Total Daily Cost ($)',
    'baseline': baseline_total_cost,
    'optimized': optimized_total_cost,
    'change': -(baseline_total_cost - optimized_total_cost),
    'percent_change': -(baseline_total_cost - optimized_total_cost)/baseline_total_cost*100
}, {
    'metric': 'Annual Cost ($)',
    'baseline': baseline_total_cost * 365,
    'optimized': optimized_total_cost * 365,
    'change': -(baseline_total_cost - optimized_total_cost) * 365,
    'percent_change': -(baseline_total_cost - optimized_total_cost)/baseline_total_cost*100
}])

print("\n" + "="*80)
print("SUMMARY STATISTICS TABLE")
print("="*80)
print(summary_stats.to_string(index=False))

# ============================================================================
# EXPORT TO FILES
# ============================================================================

print("\n" + "="*80)
print("EXPORTING RESULTS TO FILES")
print("="*80)

# Export comprehensive results
comprehensive_df.to_csv('comprehensive_stop_results.csv', index=False)
print("✓ Saved: comprehensive_stop_results.csv")

# Export summary stats
summary_stats.to_csv('summary_statistics.csv', index=False)
print("✓ Saved: summary_statistics.csv")

# Export to Excel with multiple sheets
with pd.ExcelWriter('stochastic_optimization_complete_results.xlsx', engine='openpyxl') as writer:
    # Sheet 1: Summary Statistics
    summary_stats.to_excel(writer, sheet_name='Summary_Statistics', index=False)
    
    # Sheet 2: All Stops (Comprehensive)
    comprehensive_df.to_excel(writer, sheet_name='All_Stops', index=False)
    
    # Sheet 3: Open Stops Only
    open_stops_df = comprehensive_df[comprehensive_df['status'] == 'Open'].copy()
    open_stops_df.to_excel(writer, sheet_name='Open_Stops', index=False)
    
    # Sheet 4: Closed Stops Only
    closed_stops_df = comprehensive_df[comprehensive_df['status'] == 'Closed'].copy()
    closed_stops_df.to_excel(writer, sheet_name='Closed_Stops', index=False)
    
    # Sheet 5: Stops with Increased Ridership (due to redistribution)
    increased_ridership = comprehensive_df[comprehensive_df['ridership_change'] > 0].copy()
    increased_ridership = increased_ridership.sort_values('ridership_change', ascending=False)
    increased_ridership.to_excel(writer, sheet_name='Stops_Gained_Ridership', index=False)
    
    # Sheet 6: Walk Cost Details
    walk_cost_detail = closed_stops_df[['stop_index', 'stop_id', 'stop_name', 'baseline_ridership', 
                                         'walk_distance_to_nearest', 'walk_cost_daily']].copy()
    walk_cost_detail['walk_cost_annual'] = walk_cost_detail['walk_cost_daily'] * 365
    walk_cost_detail.to_excel(writer, sheet_name='Walk_Cost_Details', index=False)

print("✓ Saved: stochastic_optimization_complete_results.xlsx (with 6 sheets)")

print("\n" + "="*80)
print("ALL RESULTS SAVED SUCCESSFULLY!")
print("="*80)

print("\nFiles created:")
print("  1. comprehensive_stop_results.csv - Stop-by-stop comparison with walk costs")
print("  2. summary_statistics.csv - Key metrics summary including walk costs")
print("  3. stochastic_optimization_complete_results.xlsx - Excel with all sheets")

print("\n" + "="*80)


STORING COMPREHENSIVE STOCHASTIC OPTIMIZATION RESULTS

Comprehensive Results DataFrame Created
Shape: (53, 17)

First 10 rows:
   stop_index   stop_id  stop_sequence                  stop_name  status  \
0           0  901736.0            1.0        LINCOLN RD/OCEAN AV    Open   
1           1  303710.0            2.0  EMPIRE BLVD/WASHINGTON AV    Open   
2           2  303711.0            3.0     EMPIRE BLVD/BEDFORD AV  Closed   
3           3  303712.0            4.0      EMPIRE BLVD/ROGERS AV    Open   
4           4  303713.0            5.0    EMPIRE BLVD/NOSTRAND AV    Open   
5           5  303714.0            6.0    EMPIRE BLVD/NEW YORK AV    Open   
6           6  306974.0            7.0    EMPIRE BLVD/BROOKLYN AV  Closed   
7           7  306322.0            8.0      EMPIRE BLVD/LAMONT CT    Open   
8           8  303717.0            9.0  KINGSTON AV/MONTGOMERY ST    Open   
9           9  303718.0           10.0     KINGSTON AV/CARROLL ST  Closed   

   baseline_ridership  o